In [2]:
import requests
import os
from dotenv import load_dotenv
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import MarianMTModel, MarianTokenizer
import faiss

In [3]:
# Load environment variables
load_dotenv()

# Groq API setup
GROQ_API_KEY = os.getenv("GROQ_API")
GROQ_API_URL = "https://api.groq.com/openai/v1/chat/completions"

In [4]:
# Function to get response from Groq API
def get_groq_response(prompt):
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "mixtral-8x7b-32768",
        "messages": [{"role": "user", "content": prompt}]
    }
    response = requests.post(GROQ_API_URL, headers=headers, json=data)
    return response.json()['choices'][0]['message']['content']

In [5]:
# Load pre-trained sentence transformer model
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

/Users/brandonamarasingam/anaconda3/envs/pytorch/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
import requests
import os
import json
from dotenv import load_dotenv
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss

# Load environment variables
load_dotenv()

# Groq API setup
GROQ_API_KEY = os.getenv("GROQ_API")
GROQ_API_URL = "https://api.groq.com/openai/v1/chat/completions"

# Load pre-trained sentence transformer model
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

# Load and preprocess Elon Musk corpus
with open('speech_total.json', 'r') as file:
    elon_corpus = json.load(file)

# Encode Elon Musk corpus
elon_embeddings = sentence_model.encode(elon_corpus)

# Create FAISS index
dimension = elon_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(elon_embeddings.astype('float32'))

def get_groq_response(prompt):
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "mixtral-8x7b-32768",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7,
        "max_tokens": 150,
        "n": 1
    }
    try:
        response = requests.post(GROQ_API_URL, headers=headers, json=data)
        response.raise_for_status()  # Raise an exception for bad status codes
        response_json = response.json()
        
        if 'choices' not in response_json:
            print("Unexpected API response structure. Full response:")
            print(json.dumps(response_json, indent=2))
            return "I'm sorry, but I couldn't generate a response at this time due to an API error."
        
        return response_json['choices'][0]['message']['content']
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while calling the Groq API: {e}")
        return "I'm sorry, but I encountered an error while trying to generate a response."

def find_similar_elon_phrases(text, top_k=5):
    query_embedding = sentence_model.encode([text])[0]
    distances, indices = index.search(np.array([query_embedding]).astype('float32'), top_k)
    return [elon_corpus[i] for i in indices[0]]

def elonify_response(original_response):
    sentences = original_response.split('.')
    elonified_sentences = []
    
    for sentence in sentences:
        if sentence.strip():
            similar_phrases = find_similar_elon_phrases(sentence)
            if similar_phrases:
                elonified_sentences.append(similar_phrases[0])
            else:
                elonified_sentences.append(sentence)
    
    return '. '.join(elonified_sentences)

def chatbot(user_input):
    groq_response = get_groq_response(user_input)
    elon_style_response = elonify_response(groq_response)
    return elon_style_response

def interactive_chat():
    print("Welcome to the Elon Musk-style Chatbot!")
    print("Type 'exit' to end the conversation.")
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == 'exit':
            print("Thank you for chatting. Goodbye!")
            break
        response = chatbot(user_input)
        print(f"Elon: {response}")

if __name__ == "__main__":
    interactive_chat()

Welcome to the Elon Musk-style Chatbot!
Type 'exit' to end the conversation.
Elon: What questions can I answer?. I read a lot of books and talked to lots of people. I didn't have any one person who was a mentor but I always looked for feedback from the people around me and feedback from the historical context, which is books basically.. So I'm gunna go into detail on all those.. I read a lot of books and talked to lots of people. I didn't have any one person who was a mentor but I always looked for feedback from the people around me and feedback from the historical context, which is books basically.. Once again, thank you for all your efforts. I could do a bit of Q&A if you'd like
Elon: Yeah, well, I mean, the point of NATO is defending Europe and it's, you know, it's like, okay, well, why is the United States paying disproportionately more to defend Europe than Europe? That doesn't make sense, that's unfair. And that is an appropriate thing to address.. If measured by launch contracts

In [ ]:

# Groq API setup
GROQ_API_KEY = "your_groq_api_key"
GROQ_API_URL = "https://api.groq.com/openai/v1/chat/completions"

# Load pre-trained sentence transformer model
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

# Load style transfer model
style_model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-en')
style_tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-en')

# Load and preprocess Elon Musk corpus
with open('elon_musk_corpus.txt', 'r') as file:
    elon_corpus = file.readlines()
elon_corpus = [sentence.strip() for sentence in elon_corpus if sentence.strip()]

# Encode Elon Musk corpus
elon_embeddings = sentence_model.encode(elon_corpus)

# Create FAISS index
dimension = elon_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(elon_embeddings.astype('float32'))

def get_groq_response(prompt):
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "mixtral-8x7b-32768",
        "messages": [{"role": "user", "content": prompt}]
    }
    response = requests.post(GROQ_API_URL, headers=headers, json=data)
    return response.json()['choices'][0]['message']['content']

def find_similar_elon_phrases(text, top_k=5):
    query_embedding = sentence_model.encode([text])[0]
    distances, indices = index.search(np.array([query_embedding]).astype('float32'), top_k)
    return [elon_corpus[i] for i in indices[0]]

def rephrase_sentence(original, style_reference):
    # Combine original and style reference for context
    input_text = f"{original} </s> {style_reference}"
    input_ids = style_tokenizer.encode(input_text, return_tensors="pt")
    
    outputs = style_model.generate(input_ids, max_length=100, num_return_sequences=1, temperature=0.7)
    rephrased = style_tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return rephrased

def elonify_response(original_response):
    sentences = original_response.split('.')
    elonified_sentences = []
    
    for sentence in sentences:
        if sentence.strip():
            similar_phrases = find_similar_elon_phrases(sentence)
            if similar_phrases:
                rephrased = rephrase_sentence(sentence, similar_phrases[0])
                elonified_sentences.append(rephrased)
            else:
                elonified_sentences.append(sentence)
    
    return '. '.join(elonified_sentences)

def chatbot(user_input):
    groq_response = get_groq_response(user_input)
    elon_style_response = elonify_response(groq_response)
    return elon_style_response

# Example usage
user_question = "What do you think about the future of electric vehicles?"
response = chatbot(user_question)
print(response)